# DO NACs

This file demonstrates how to run the calculations of the NACs in the KS space, using QE.

In particular, this example is designed to run the calculations in a serial fashion - without scheduling procedures (e.g. on a personal laptop). This is quire useful for debugging and development purposes.

So, lets start by loading the required modules

In [1]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
    

from libra_py import hpc_utils
from libra_py import data_read
from libra_py import data_outs
from libra_py import units
from libra_py import QE_methods
from libra_py.workflows.nbra import step2
#import libra_py.workflows.nbra.step2_analysis as step2a

import numpy as np
#from matplotlib.mlab import griddata

import matplotlib.pyplot as plt   # plots
%matplotlib inline 

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, s

Assume we have already produced a QE MD trajectory and it is stored in the file **x0.md.out** (which we copied in the present directory).

We need to also create a **x0.scf.in** file that contains the parameters for QE calculations (the type of calculation should be *scf*). The file should not contain the atomic coordiantes section, but should contain the cell parameters sections or occupations if they are used. Place the file in the current directory.

Also create a **x0.exp.in** file (also to be placed in the present directory). It shall describe the procedured for the wavefunction "export" operation - mainly the location and names of atomic pseudopotentials and the correct prefix for the files. 

The following section will clean up the previous results and temporary directory (BEWARE!!! you may not always want to do this for it will delete expensive results)

In [2]:
# Remove the previous results and temporary working directory from the previous runs
os.system("rm -r res")
os.system("rm -r wd")

# Create the new results directory
os.system("mkdir res")
rd = os.getcwd()+"/res"          # where all the results stuff will go

We'll use **QE_methods.out2inp()** function to convert the MD trajectory into a bunch of input files for SCF calculation - this is something we'll need for NAC calculations. 

In this case, you need to setup the *iinit* and *ifinal* variables which determine which steps of the original MD trajectory will be used to produce the input files and subsequenctly used in the NACs calculations

All these files will be generated in the temporarily-created **wd** directory. The system then "cd" into that directory to start the consecutive operations in that directory.

In [3]:
iinit = 0
ifinal = 10

QE_methods.out2inp("x0.md.out","x0.scf.in","wd","x0.scf", iinit, ifinal,1)
os.system("cp x0.exp.in wd"); 

Next - define the parameters used by the **step2.run()** (and other functions called inside, so look for the description all all suitable parameters). The meaning of the most parameters is quite intuitive. Let me just clarify a couple less-obvious points:

In this example, I do the calculations directly, so without "parallelizing" via the batch system, so specify "BATCH_SYSTEM":None

The system in this example has 56 electrons, so the HOMO would correspond to number 28 and LUMO to number 29. In this example, we use more orbitals - those below HOMO and above LUMO: minband = 20 and maxband = 39, so we could study the HOMO-LUMO transitions as well as all other types of relaxation. In all, there are 20 orbitals included in our present active space. The resulting files will contain matrixes 40 by 40 - because we plot alpha and beta channels. Just a reminder: the orbital indexing starts from 1.

In [4]:
params = { "EXE"       :"/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw.x", 
           "EXE_EXPORT":"/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw_export.x",
           "BATCH_SYSTEM":None,
           "start_indx":0, "stop_indx":10, "dt":1.0*units.fs2au, 
           "prefix0":"x0.scf", 
           "nac_method":1, 
           "minband":20, "maxband":39, 
           "minband_soc":20, "maxband_soc":39, 
           "compute_Hprime":True, 
           "wd":"wd",
           "rd":rd,
           "verbosity":0
         }

os.chdir("wd")
step2.run(params)
os.chdir("../")

The time to run the QE calculations =  0.0625
The time to run the QE calculations =  0.09375
path= Root
The total # of k-points (spin-polarized) including up and down components is:  2
The time to get the basic parameters about your QE calculations =  0.0
printing prefix:   curr0/x0.export
Reading index from file  wd/curr0/x0.export/index.xml
path= Root
 nspin =  2  nk =  2  nbnd =  40  efermi =  0.10401786287043  alat =  1.89  omega =  1089.429480162654  tpiba =  3.32443667046539  tpiba2 =  3.32443667046539
 Direct lattice vectors: 
 a1 =  8.91510449256 -0.02931884235 -0.00453723984
 a2 =  -4.482941481899999 7.678748407050002 -0.0005173497
 a3 =  -0.004824402660000001 -0.00262325385 15.9447227247
 Reciprocal lattice vectors: 
 b1 =  0.7061356696551127 0.4122501302508279 0.0002814799367690439
 b2 =  0.002696221652374108 0.8198305306489484 0.0001356957591637749
 b3 =  0.0002010258715444394 0.0001439107367195997 0.3940605780990116
 K points: 
0  weight =  1.0  k =  0.0 0.0 0.0
1  weight 

path= Kpoint.1
path= Kpoint.1
Reading the grid from file  wd/next0/x0.export/grid.1
path= Kpoint.1
Handling the k-point 1 with coordinates:  0.00000  0.00000  0.00000 
Reading the wfc from file  wd/next0/x0.export/wfc.2
path= Kpoint.2
path= Kpoint.2
Reading the grid from file  wd/next0/x0.export/grid.2
path= Kpoint.2
The time to read index, wavefunctions, and grid about your QE calculations =  0.703125
Time to read index, wfc, and wfc grids =  1.484375
The time to run the QE calculations =  0.078125
path= Root
The total # of k-points (spin-polarized) including up and down components is:  2
The time to get the basic parameters about your QE calculations =  0.0
printing prefix:   curr0/x0.export
Reading index from file  wd/curr0/x0.export/index.xml
path= Root
 nspin =  2  nk =  2  nbnd =  40  efermi =  0.10395603338417  alat =  1.89  omega =  1089.429480162654  tpiba =  3.32443667046539  tpiba2 =  3.32443667046539
 Direct lattice vectors: 
 a1 =  8.91510449256 -0.02931884235 -0.004537239

path= Kpoint.1
path= Kpoint.1
Reading the grid from file  wd/next0/x0.export/grid.1
path= Kpoint.1
Handling the k-point 1 with coordinates:  0.00000  0.00000  0.00000 
Reading the wfc from file  wd/next0/x0.export/wfc.2
path= Kpoint.2
path= Kpoint.2
Reading the grid from file  wd/next0/x0.export/grid.2
path= Kpoint.2
The time to read index, wavefunctions, and grid about your QE calculations =  1.0625
Time to read index, wfc, and wfc grids =  1.8125
The time to run the QE calculations =  0.078125
path= Root
The total # of k-points (spin-polarized) including up and down components is:  2
The time to get the basic parameters about your QE calculations =  0.015625
printing prefix:   curr0/x0.export
Reading index from file  wd/curr0/x0.export/index.xml
path= Root
 nspin =  2  nk =  2  nbnd =  40  efermi =  0.1039597083243428  alat =  1.89  omega =  1089.429480162654  tpiba =  3.32443667046539  tpiba2 =  3.32443667046539
 Direct lattice vectors: 
 a1 =  8.91510449256 -0.02931884235 -0.004537

path= Kpoint.1
Reading the grid from file  wd/next0/x0.export/grid.1
path= Kpoint.1
Handling the k-point 1 with coordinates:  0.00000  0.00000  0.00000 
Reading the wfc from file  wd/next0/x0.export/wfc.2
path= Kpoint.2
path= Kpoint.2
Reading the grid from file  wd/next0/x0.export/grid.2
path= Kpoint.2
The time to read index, wavefunctions, and grid about your QE calculations =  0.71875
Time to read index, wfc, and wfc grids =  1.5
The time to run the QE calculations =  0.078125
path= Root
The total # of k-points (spin-polarized) including up and down components is:  2
The time to get the basic parameters about your QE calculations =  0.0
printing prefix:   curr0/x0.export
Reading index from file  wd/curr0/x0.export/index.xml
path= Root
 nspin =  2  nk =  2  nbnd =  40  efermi =  0.1039905852799617  alat =  1.89  omega =  1089.429480162654  tpiba =  3.32443667046539  tpiba2 =  3.32443667046539
 Direct lattice vectors: 
 a1 =  8.91510449256 -0.02931884235 -0.00453723984
 a2 =  -4.482941